<a href="https://colab.research.google.com/github/lorenzopaoria/Smoking-detection-and-distance-analysis/blob/main/model_train_person.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Train a model for sigarette detection

In [62]:
pip install ultralytics

In [63]:
from ultralytics import YOLO
import torch
from pathlib import Path
import json
from datetime import datetime
import psutil
from tqdm.auto import tqdm

In [64]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [65]:
class ModelCallback:
    def __init__(self, save_dir, epochs):
        self.save_dir = Path(save_dir)
        self.save_dir.mkdir(parents=True, exist_ok=True)
        self.best_map = 0
        self.training_history = []
        self.epochs_pbar = tqdm(total=epochs, desc="Epochs", position=0)
        self.batch_pbar = tqdm(total=100, desc="Batches", position=1, leave=False, bar_format='{l_bar}{bar:10}{r_bar}')
        self.current_epoch = 0

    def on_train_batch_end(self, trainer):
        if self.current_epoch != trainer.epoch:
            self.batch_pbar.reset()
            self.current_epoch = trainer.epoch
            if hasattr(trainer, 'nb_batches'):
                self.batch_pbar.total = trainer.nb_batches

        # Aggiorna la barra senza andare a capo
        self.batch_pbar.update(1)
        self.batch_pbar.refresh()

    def on_train_epoch_end(self, trainer):
        metrics = trainer.metrics
        epoch = trainer.epoch
        current_map = metrics.get('metrics/mAP50-95(B)', 0)

        # Esegui validazione
        val_results = trainer.model.val(verbose=False)  # Disabilita output verboso
        val_map = val_results.box.map
        val_map50 = val_results.box.map50

        self.training_history.append({
            'epoch': epoch,
            'train_mAP': current_map,
            'val_mAP': val_map,
            'val_mAP50': val_map50,
            'metrics': metrics
        })

        with open(self.save_dir / 'training_history.json', 'w') as f:
            json.dump(self.training_history, f, indent=4)

        # Salva il modello se la mAP di validazione è migliore
        if val_map > self.best_map:
            self.best_map = val_map
            best_path = self.save_dir / f'best_model_map_{val_map:.4f}.pth'
            torch.save({
                'epoch': epoch,
                'model_state_dict': trainer.model.state_dict(),
                'val_map': val_map,
                'val_map50': val_map50,
                'train_map': current_map
            }, best_path)

        # Salva l'ultimo modello
        last_path = self.save_dir / 'last_model.pth'
        torch.save({
            'epoch': epoch,
            'model_state_dict': trainer.model.state_dict(),
            'val_map': val_map,
            'val_map50': val_map50,
            'train_map': current_map
        }, last_path)

        # Aggiorna la barra delle epoche
        self.epochs_pbar.update(1)
        self.epochs_pbar.set_postfix({
            'train_mAP': f'{current_map:.4f}',
            'val_mAP': f'{val_map:.4f}'
        }, refresh=True)

    def on_train_end(self, trainer):
        self.epochs_pbar.close()
        self.batch_pbar.close()

In [ ]:
def train_model(yaml_path, save_dir, epochs=100, imgsz=1024, batch=16):
    model = ('yolov10n.pt')
    callback = ModelCallback(save_dir, epochs)

    results = model.train(
        data=yaml_path,
        epochs=epochs,
        imgsz=imgsz,
        batch=batch,
        device=0 if torch.cuda.is_available() else 'cpu',
        patience=6,
        save=True,
        project=str(save_dir),
        name='train',
        exist_ok=True,
        val=True,
        verbose=False  # Disabilita output verboso
    )

    return model, callback.best_map

In [ ]:
if __name__ == "__main__":
    yaml_path = '/content/drive/MyDrive/Photo person/data.yaml'
    save_dir = Path('/content/drive/MyDrive/pth_person_detect')
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    run_dir = save_dir / f'run_{timestamp}'

    model, best_map = train_model(yaml_path, run_dir, epochs=50)
    print(f"\nTraining completed!")
    print(f"Best mAP: {best_map:.4f}")
    print(f"Models saved in: {run_dir}")

Epochs:   0%|          | 0/10 [00:00<?, ?it/s]

Batches:   0%|          | 0/100 [00:00<?, ?it/s]

Ultralytics 8.3.74 🚀 Python-3.11.11 torch-2.5.1+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/content/drive/MyDrive/Photo person/data.yaml, epochs=10, time=None, patience=6, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=0, workers=8, project=/content/drive/MyDrive/pth_person_detect/run_20250211_095730, name=train, exist_ok=True, pretrained=True, optimizer=auto, verbose=False, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_con



train: Scanning /content/drive/MyDrive/Photo person/train/labels.cache... 788 images, 0 backgrounds, 0 corrupt: 100%|██████████| 788/788 [00:00<?, ?it/s]

albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, num_output_channels=3, method='weighted_average'), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))


...



  0%|          | 0/50 [00:00<?, ?it/s]

      10/10      2.34G     0.3533     0.4704     0.7939        100        640:   0%|          | 0/50 [00:00<?, ?it/s]

      10/10      2.34G     0.3533     0.4704     0.7939        100        640:   2%|▏         | 1/50 [00:00<00:14,  3.29it/s]

      10/10      2.34G     0.3773     0.5268     0.8024        107        640:   2%|▏         | 1/50 [00:00<00:14,  3.29it/s]

      10/10      2.34G     0.3773     0.5268     0.8024        107        640:   4%|▍         | 2/50 [00:00<00:15,  3.18it/s]

      10/10      2.34G     0.3799     0.5494      0.804        104        640:   4%|▍         | 2/50 [00:01<00:15,  3.18it/s]

      10/10      2.34G     0.3799     0.5494      0.804        104        640:   6%|▌         | 3/50 [00:01<00:16,  2.78it/s]

      10/10      2.34G     0.3755     0.5441     0.8111        105        640:   6%|▌         | 3/50 [00:01<00:16,  2.78it/s]

      10/10      2.34G     0.3755     0.5441     0.8111        105        640

                   all        151        999      0.777      0.764      0.781      0.668



10 epochs completed in 0.095 hours.
Optimizer stripped from /content/drive/MyDrive/pth_person_detect/run_20250211_095730/train/weights/last.pt, 6.2MB
Optimizer stripped from /content/drive/MyDrive/pth_person_detect/run_20250211_095730/train/weights/best.pt, 6.2MB

Validating /content/drive/MyDrive/pth_person_detect/run_20250211_095730/train/weights/best.pt...
Ultralytics 8.3.74 🚀 Python-3.11.11 torch-2.5.1+cu124 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 168 layers, 3,006,233 parameters, 0 gradients, 8.1 GFLOPs




                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95):   0%|          | 0/5 [00:00<?, ?it/s]

                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95):  20%|██        | 1/5 [00:00<00:01,  3.26it/s]

                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95):  40%|████      | 2/5 [00:01<00:02,  1.43it/s]

                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95):  60%|██████    | 3/5 [00:03<00:02,  1.17s/it]

                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95):  80%|████████  | 4/5 [00:04<00:01,  1.34s/it]

                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 5/5 [00:05<00:00,  1.15s/it]


                   all        151        999      0.778      0.763      0.781      0.668
Speed: 0.6ms preprocess, 4.0ms inference, 0.0ms loss, 7.0ms postprocess per image
Results saved to /content/drive/MyDrive/pth_person_detect/run_20250211_095730/train

Training completed!
Best mAP: 0.0000
Models saved in: /content/drive/MyDrive/pth_person_detect/run_20250211_095730
